In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

import math

/home/karlositalo/miniconda2/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
X = pd.read_csv("dados/train_images.csv", header=None)
y = pd.read_csv("dados/train_labels.csv")["Volcano?"]
X_test = pd.read_csv("dados/test_images.csv", header=None)
y_test = pd.read_csv("dados/test_labels.csv")["Volcano?"]

In [3]:
print(X.shape)
print(y.shape)

print(X_test.shape)
print(y_test.shape)

(7000, 12100)
(7000,)
(2734, 12100)
(2734,)


In [4]:
print(X.head())
print(y.head())

print(X_test.head())
print(y_test.head())

   0      1      2      3      4      5      6      7      8      9      \
0     95    101     99    103     95     86     96     89     70    104   
1     91     92     91     89     92     93     96    101    107    104   
2     87     70     72     74     84     78     93    104    106    106   
3      0      0      0      0      0      0      0      0      0      0   
4    114    118    124    119     95    118    105    116    123    112   

   ...    12090  12091  12092  12093  12094  12095  12096  12097  12098  12099  
0  ...      111    107     92     89    103     99    117    116    118     96  
1  ...      103     92     93     95     98    105    104    100     90     81  
2  ...       84     71     95    102     94     80     91     80     84     90  
3  ...       94     81     89     84     80     90     92     80     88     96  
4  ...      116    113    102     93    109    104    106    117    111    115  

[5 rows x 12100 columns]
0    1
1    0
2    0
3    0
4    0
Na

In [19]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.fit_transform(X_test)

In [5]:
classifier = SVC(kernel = 'linear' , random_state = 0)
classifier.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=0,
  shrinking=True, tol=0.001, verbose=False)

In [6]:
predictions = classifier.predict(X_test)
print(classification_report(y_test,predictions))
print("Score: {}".format(accuracy_score(y_test, predictions)))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      2300
           1       0.78      0.66      0.72       434

   micro avg       0.92      0.92      0.92      2734
   macro avg       0.86      0.81      0.83      2734
weighted avg       0.91      0.92      0.91      2734

Score: 0.917337234821


In [10]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2000)
pca.fit(X)
X_pca = pca.transform(X)
X_test_pca = pca.transform(X_test)

In [14]:
print(X_pca.shape)
print(X_test_pca.shape)

(7000, 2000)
(2734, 2000)


In [11]:
classifier_pca = SVC(kernel = 'linear' , random_state = 0)
classifier_pca.fit(X_pca,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=0,
  shrinking=True, tol=0.001, verbose=False)

In [15]:
predictions_pca = classifier_pca.predict(X_test_pca)
print(classification_report(y_test,predictions_pca))
print("Score: {}".format(accuracy_score(y_test, predictions_pca)))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2300
           1       0.77      0.67      0.72       434

   micro avg       0.92      0.92      0.92      2734
   macro avg       0.86      0.82      0.83      2734
weighted avg       0.91      0.92      0.91      2734

Score: 0.916239941478


In [16]:
classifier_opt = SVC(kernel = 'linear' , C=1e-06)
classifier_opt.fit(X_pca,y)

SVC(C=1e-06, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [17]:
predictions_pca_opt = classifier_opt.predict(X_test_pca)
print(classification_report(y_test,predictions_pca_opt))
print("Score: {}".format(accuracy_score(y_test, predictions_pca_opt)))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2300
           1       0.88      0.66      0.75       434

   micro avg       0.93      0.93      0.93      2734
   macro avg       0.91      0.82      0.86      2734
weighted avg       0.93      0.93      0.93      2734

Score: 0.931967812729


In [20]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            #np.random.shuffle(this_xs)
            this_xs = this_xs.reindex(np.random.permutation(this_xs.index))

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    #xs = np.concatenate(xs)
    #ys = np.concatenate(ys)
    xs = pd.concat(xs)
    ys = pd.Series(data=np.concatenate(ys),name='target')

    return xs,ys

In [24]:
xb,yb = balanced_subsample(X,y);
#xb_pca,_ = balanced_subsample(X_pca,y);

In [25]:
print(xb.shape)
print(yb.shape)

print(X.shape)
print(y.shape)

(2000, 12100)
(2000,)
(7000, 12100)
(7000,)


In [32]:
classifier2 = SVC(kernel = 'linear' , random_state = 0)
classifier2.fit(xb,yb)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=0,
  shrinking=True, tol=0.001, verbose=False)

In [33]:
predictions2 = classifier2.predict(X_test)
print(classification_report(y_test,predictions2))
print("Score: {}".format(accuracy_score(y_test, predictions2)))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      2300
           1       0.49      0.84      0.62       434

   micro avg       0.83      0.83      0.83      2734
   macro avg       0.73      0.84      0.75      2734
weighted avg       0.89      0.83      0.85      2734

Score: 0.833211411851


In [33]:
classifier3 = SVC(kernel = 'linear')
classifier3.fit(xb,yb)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [34]:
predictions3 = classifier3.predict(X_test)
print(classification_report(y_test,predictions3))
print("Score: {}".format(accuracy_score(y_test, predictions3)))

             precision    recall  f1-score   support

          0       0.96      0.85      0.91      2300
          1       0.52      0.83      0.64       434

avg / total       0.89      0.85      0.86      2734

Score: 0.8507681053401609


In [36]:
classifier4 = SVC(kernel = 'linear',C = 0.1)
classifier4.fit(xb,yb)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [37]:
predictions4 = classifier4.predict(X_test)
print(classification_report(y_test,predictions4))
print("Score: {}".format(accuracy_score(y_test, predictions4)))

             precision    recall  f1-score   support

          0       0.96      0.85      0.91      2300
          1       0.52      0.83      0.64       434

avg / total       0.89      0.85      0.86      2734

Score: 0.8507681053401609


In [39]:
classifier5 = SVC(kernel = 'rbf')
classifier5.fit(xb,yb)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [40]:
predictions5 = classifier5.predict(X_test)
print(classification_report(y_test,predictions5))
print("Score: {}".format(accuracy_score(y_test, predictions5)))

             precision    recall  f1-score   support

          0       0.96      0.92      0.94      2300
          1       0.66      0.79      0.72       434

avg / total       0.91      0.90      0.91      2734

Score: 0.9019751280175567


In [41]:
classifier6 = SVC(kernel = 'poly')
classifier6.fit(xb,yb)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
predictions6 = classifier6.predict(X_test)
print(classification_report(y_test,predictions6))
print("Score: {}".format(accuracy_score(y_test, predictions6)))

             precision    recall  f1-score   support

          0       0.92      0.88      0.90      2300
          1       0.48      0.59      0.53       434

avg / total       0.85      0.84      0.84      2734

Score: 0.8354059985369422


In [43]:
classifier7 = SVC(kernel = 'sigmoid')
classifier7.fit(xb,yb)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
predictions7 = classifier7.predict(X_test)
print(classification_report(y_test,predictions7))
print("Score: {}".format(accuracy_score(y_test, predictions7)))

             precision    recall  f1-score   support

          0       0.84      0.51      0.63      2300
          1       0.16      0.49      0.24       434

avg / total       0.73      0.51      0.57      2734

Score: 0.5058522311631309


In [46]:
classifier8 = SVC(kernel = "rbf")
classifier8.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
predictions8 = classifier8.predict(X_test)
print(classification_report(y_test,predictions8))
print("Score: {}".format(accuracy_score(y_test, predictions8)))

             precision    recall  f1-score   support

          0       0.90      1.00      0.95      2300
          1       0.97      0.42      0.59       434

avg / total       0.91      0.91      0.89      2734

Score: 0.9059985369422092


In [48]:
print(X.shape)
print(y.shape)

(7000, 12100)
(7000,)


In [28]:
classifier9 = SVC(kernel = 'linear' , C=1.1)
classifier9.fit(xb,yb)

SVC(C=1.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [29]:
predictions9 = classifier9.predict(X_test)
print(classification_report(y_test,predictions9))
print("Score: {}".format(accuracy_score(y_test, predictions9)))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      2300
           1       0.49      0.84      0.62       434

   micro avg       0.83      0.83      0.83      2734
   macro avg       0.73      0.84      0.75      2734
weighted avg       0.89      0.83      0.85      2734

Score: 0.833211411851


In [30]:
classifier10 = SVC(kernel = 'linear' , C=0.9)
classifier10.fit(xb,yb)

SVC(C=0.9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [31]:
predictions10 = classifier10.predict(X_test)
print(classification_report(y_test,predictions10))
print("Score: {}".format(accuracy_score(y_test, predictions10)))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      2300
           1       0.49      0.84      0.62       434

   micro avg       0.83      0.83      0.83      2734
   macro avg       0.73      0.84      0.75      2734
weighted avg       0.89      0.83      0.85      2734

Score: 0.833211411851


In [34]:
classifier11 = SVC(kernel = 'linear' , C=1.5)
classifier11.fit(xb,yb)

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [35]:
predictions11 = classifier11.predict(X_test)
print(classification_report(y_test,predictions11))
print("Score: {}".format(accuracy_score(y_test, predictions11)))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      2300
           1       0.49      0.84      0.62       434

   micro avg       0.83      0.83      0.83      2734
   macro avg       0.73      0.84      0.75      2734
weighted avg       0.89      0.83      0.85      2734

Score: 0.833211411851


In [36]:
classifier12 = SVC(kernel = 'linear' , C=0.5)
classifier12.fit(xb,yb)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [37]:
predictions12 = classifier12.predict(X_test)
print(classification_report(y_test,predictions12))
print("Score: {}".format(accuracy_score(y_test, predictions12)))

              precision    recall  f1-score   support

           0       0.97      0.83      0.89      2300
           1       0.49      0.84      0.62       434

   micro avg       0.83      0.83      0.83      2734
   macro avg       0.73      0.84      0.75      2734
weighted avg       0.89      0.83      0.85      2734

Score: 0.833211411851
